# Biol 359A  | Complexity, Overfitting, and Model Selection
### Spring 2023, Week 7
<hr>

Objectives:
-  Develop and run increasingly complex MLR models
-  Evaluate performance of MLR models based on Cross-Validation
-  Evaluate other supervised learning models based on Cross-Validation

# Let us discuss the dataset and the problem: 

Imagine you are a want to leverage small interfering RNAs (siRNA) to adjust the transcription rate of a gene. 
The thermodynamics (a characterization of equilibrium energy) of these structures is closely tied to how accessible the DNA is for transcription (as well as any other biomolecule _complex_ that you may encounter). 
The main property we care about here is the Gibbs Free Energy, $\Delta G$. 

$\Delta G$ describes the strength of localization between two molecules. The more negative the $\Delta G$, the more energy it will take to separate these molecules (alternatively, the more likely these molecules will form a complex). When considering multiple complexes that can form, its often useful to use this to evaluate the __competition__ between which complexes will form. These concepts are closely tied with proability and statistics. 

Here is the competition system we are studying:


![](system.png)

Therefore, a lower activity means that we have a more effective inhibition.

Source: https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-7-65

In [ ]:
!git clone https://github.com/BIOL359A-FoundationsOfQBio-Spr23/week7_modelselection
!mkdir ./data
!cp week7_modelselection/data/* ./data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import sklearn as sk
import matplotlib.pyplot as plt
import ipywidgets as widgets

from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, cross_validate

%matplotlib inline


TITLE_FONT = 20
LABEL_FONT = 16
TICK_FONT = 16
FIG_SIZE = (5,5)
COLORS= ["#008080","#CA562C"]

sns.set(font_scale=1.5, rc={'figure.figsize':FIG_SIZE}) 
sns.set_style("whitegrid",  {'axes.linewidth': 2, 'axes.edgecolor':'black'})
plt.rc("axes.spines", top=False, right=False)


| Feature Name | Description | 
| --- | --- |
| Target seq | target mRNA GenBank sequence accession number |
| Start | start position on target mRNA | 
| End | end position on target mRNA | 
| Sequence | siRNA sequence |
| G | nucleotide content, G |
| U | nucleotide content, U | 
| bi | stability (∆G) of dimers of siRNAs antisense strands |  
| uni | siRNA antisense strand intra-molecular structure stability (∆G) |
| duplex | ∆G of sense-antisense siRNA duplexes | 
| Pos1 | stability profile (∆G) of each two neighboring base pairs in the siRNA sense-antisense, position 1 | 
| Pos2 | stability profile (∆G) of each two neighboring base pairs in the siRNA sense-antisense, position 2 | 
| Pos6 | stability profile (∆G) of each two neighboring base pairs in the siRNA sense-antisense, position 6 | 
| Pos13 | stability profile (∆G) of each two neighboring base pairs in the siRNA sense-antisense, position 13 | 
| Pos14 | stability profile (∆G) of each two neighboring base pairs in the siRNA sense-antisense, position 14 | 
| Pos18 | stability profile (∆G) of each two neighboring base pairs in the siRNA sense-antisense, position 18 | 
| Dif_5-3 | ∆G difference between position 1 and 18 |
| Content+ | preferred dinucleotide content |
| Content- | avoided dinucleotide content | 
| Cons+ | position-dependent nucleotide consensus, preferred |
| Cons- | position-dependent nucleotide consensus, avoided |
| Cons_Sum | position-dependent nucleotide consensus, sum |
| Hyb19 | number of potential target copies in mRNAs (∆G threshold) |
| target | local target mRNA stabilities (∆G) |
| Activity | gene expression |

In [ ]:
dataset = pd.read_csv("data/data.csv")
dataset

# Question 2-5: Dataset
We are going to take out the non-numeric features. There are a couple thigns that we could potentially do with them, but that type of feature engineering is outside the scope of this class. __Refer to the table below to answer questions 2 and 3.__

In [ ]:
def get_numerical(df):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_df = df.select_dtypes(include=numerics)
    return num_df
    
num_dataset = get_numerical(dataset)
num_dataset

In [ ]:
num_dataset.describe().T

# Question 6: Training Data and Test Data

When viewing datasets, you can treat every feature as a __dimension__ (think about how each feature can be represented on an axis). Below are PCA (Principal Component Analysis) plots, which are a way to visualize high-dimensional datasets in fewer dimensions. We will cover this analysis later in the course, but if points are close on a PCA plot, it generally means they contain similar information.

As a researcher, you may need to take a look at your training/test split to see if you are including any bias in your training/test data. From class, Prof. Bagheri gave the example of your training data containing all of the placebo patients and none of them ending up in the training set, which leads to unbalanced data. 

Below are 4 different PCA plots, with training-test splits. Which are good? Which are not good? What type of bias would you introduce? 

In [ ]:

    
def performPCA(X, n_dimensions=2, drop_columns=["Start", "End"]):
    """
    Uses sklearn PCA tool to perform PCA
    input:
    X: Pandas Dataframe or Numpy Array of features
    n_dimensions: Number of PCs to fit
    
    output:
    X_pca: Pandas dataframe with column titles of PC1,...,PCn
    """
    X_data = get_numerical(X)
    if drop_columns: X_data.drop([], axis=1)
    X_standardized = StandardScaler().fit_transform(X_data)
    pca = PCA(n_components=n_dimensions)
    pca.fit(X_standardized)
    X_pca_array = pca.transform(X_standardized)
    column_names = ['PC{}'.format(i+1) for i in range(n_dimensions)] 
    X_pca = pd.DataFrame(X_pca_array, columns=column_names)
    return X_pca

def makeplots(pca_data):
    sns.scatterplot(x="PC1", y="PC2", data=pca_data, color="grey",alpha=.3, label="Training")
    biased_A = pca_data[pca_data["PC1"]>-1]
    sns.scatterplot(x="PC1", y="PC2", data=biased_A.sample(130), color="blue", label="Test")
    plt.title("A: ($n_{test}=130$)", fontweight="bold")
    plt.show()
    sns.scatterplot(x="PC1", y="PC2", data=pca_data, color="grey",alpha=.3, label="Training")
    biased_B = pca_data[pca_data["PC2"]<1.5]
    sns.scatterplot(x="PC1", y="PC2", data=biased_B.sample(130), color="orange", label="Test")
    plt.title("B: ($n_{test}=130$)", fontweight="bold")
    plt.show()
    sns.scatterplot(x="PC1", y="PC2", data=pca_data, color="grey",alpha=.3, label="Training")
    sns.scatterplot(x="PC1", y="PC2", data=pca_data.sample(130), color="purple", label="Test")
    plt.title("C: ($n_{test}=130$)", fontweight="bold")
    plt.show()
    sns.scatterplot(x="PC1", y="PC2", data=pca_data, color="grey",alpha=.3, label="Training")
    sns.scatterplot(x="PC1", y="PC2", data=pca_data.sample(40), color="red", label="Test")
    plt.title(r"D: ($n_{test}=40$)", fontweight="bold")
    plt.show()

pca = performPCA(dataset)
makeplots(pca)


# Question 7: MLR review

We will perform an MLR on __all__ of the quantitative variables in the dataset. Try to interpret the variables. Does anything seem odd? Compare the coefficients to the original ranges and typical values of the features they are associated with.

In [ ]:
def linear_regression(df, feature_cols, response_col, standardized = True, print_coef=True):
    """
    Use linear_model to run a linear regression using sklearn
    
    """
    X = df[feature_cols]
    y = df[response_col]
    if standardized:
        X = StandardScaler().fit_transform(X)
        y = StandardScaler().fit_transform(y.values.reshape(-1, 1))
    regression = linear_model.LinearRegression() 
    regression.fit(X,y)
    if print_coef:
        try:
            print('Intercept of MLR model is {0:0.2f}'.format(regression.intercept_))
        except TypeError:
            print('Intercept of MLR model is {0:0.2f}'.format(regression.intercept_[0]))
        print('Regression Coefficients: ')
        for feature, coef in zip(feature_cols, regression.coef_.flatten()):
            print(f'{feature} ~ {coef:.2f}')
    return regression.predict(X), regression.score(X,y)

def parity_plot(true, pred, r_squared=None, title='', alpha=None, color=None, hue=None):
    """
    plot true vs the predicted data
    inputs: 2 list-like (arrays) data structures
    """
    fig, ax = plt.subplots(1,1,figsize=(10, 8))
    if hue is not None:
        sns.scatterplot(x=true, y=pred, hue=hue)
    else: 
        if color is None: sns.scatterplot(x=true, y=pred)
        else: sns.scatterplot(x=true, y=pred, alpha=alpha, color=color)
    min_value = min(min(true), min(pred))
    max_value = max(max(true), max(pred))
    plt.plot([min_value, max_value],[min_value, max_value], '--', label="parity")
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    sns.despine()
    plt.text(1.01, 0.98, r"$R^2 = {0:.2f}$".format(r_squared),
         ha='left', va='top', size =LABEL_FONT,
         transform=ax.transAxes)
    plt.title('Parity Plot: {}'.format(title), size=TITLE_FONT)
    plt.legend(loc='best')
    plt.show()    
    
def run_regression(data, 
                   feature_cols = ['Start', 
                                   'End', 
                                   'G', 
                                   'U', 
                                   'bi', 
                                   'uni', 
                                   'duplex', 
                                   'Pos1', 
                                   'Pos2', 
                                   'Pos6', 
                                   'Pos13', 
                                   'Pos14', 
                                   'Pos18', 
                                   'Dif_5-3', 
                                   'Content+', 
                                   'Content-', 
                                   'Cons+', 
                                   'Cons-', 
                                   'Cons_Sum', 
                                   'Hyb19', 
                                   'target'
                                  ], 
                     response_col='Activity',
                     standardized=False,
                     parity=True,
                     ):
    y_pred, r_squared = linear_regression(data, feature_cols, response_col, standardized = standardized)
    if parity: parity_plot(data[response_col], y_pred.flatten(), r_squared)

run_regression(data=num_dataset)



# Question 8: Build your own MLR model

Let's remove some of the problematic features from the previous model. Notice how the coefficient of determination changes as you remove features. 

In [ ]:
@widgets.interact()   
def regression_wrapper(Start = True,
                       End = True,
                       G = True,
                       U = True,
                       bi = True,
                       uni = True,
                       duplex = True,
                       Pos1 = True,
                       Pos2 = True,
                       Pos6 = True,
                       Pos13 = True,
                       Pos14 = True,
                       Pos18 = True,
                       Dif_5_3 = True,
                       Content_plus = True,
                       Content_minus = True,
                       Cons_plus = True,
                       Cons_minus = True,
                       Cons_Sum = True,
                       Hyb19 = True,
                       target = True):
    response="Activity"
    features = []
    if Start: features.append('Start')
    if End: features.append('End')
    if G: features.append('G')
    if U: features.append('U')
    if bi: features.append('bi')
    if uni: features.append('uni')
    if duplex: features.append('duplex')
    if Pos1: features.append('Pos1')
    if Pos2: features.append('Pos2')
    if Pos6: features.append('Pos6')
    if Pos13: features.append('Pos13')
    if Pos14: features.append('Pos14')
    if Pos18: features.append('Pos18')
    if Dif_5_3: features.append('Dif_5-3')
    if Content_plus: features.append('Content+')
    if Content_minus: features.append('Content-')
    if Cons_plus: features.append('Cons+')
    if Cons_minus: features.append('Cons-')
    if Cons_Sum: features.append('Cons_Sum')
    if Hyb19: features.append('Hyb19')
    if target: features.append('target')        

    run_regression(data=num_dataset, feature_cols=features, response_col = response)


# Question 9-10: You are the quantitative biologist now

Go through the process of building the model. You can increase the degrees of your polynomial features, and include interaction terms (covariance) in order to build the model. Notice how the fit of the model changes if you include or don't include a particular value. 

Try to do this without selecting `test` (final option). Use `test` once you're comfortable with the model performance on validation data. 

In [ ]:
def cross_validation(model, X, y, k=10):
    scores = cross_validate(model, X, y, cv=k,
                            scoring=('r2', 'neg_root_mean_squared_error'),
                            return_train_score=True)
    
    print(f"Training RMSE: {np.mean(-1*scores['train_neg_root_mean_squared_error']):.2f}")
    print(f"Validation RMSE: {np.mean(-1*scores['test_neg_root_mean_squared_error']):.2f}")
    
    print(f"Training R2: {np.mean(scores['train_r2']):.3f}")
    print(f"Validation R2: {np.mean(scores['test_r2']):.3f}")
    
    return model.fit(X,y)
    
def do_the_thing(X,y , degrees, interactions, test):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    poly = PolynomialFeatures(degrees, include_bias=True)
    # print(poly.get_feature_names_out(input_features=X_train.columns))
    X_train = poly.fit_transform(X_train)
    poly_features = poly.get_feature_names()
    X_test = poly.fit_transform(X_test, poly_features)
    X_train_df = pd.DataFrame(X_train, columns=poly_features)
    X_test_df = pd.DataFrame(X_test, columns=poly_features)
    interaction_list = [feat for feat in poly_features if len(feat.split())!=1]
    if not interactions:
        X_train_df = X_train_df.drop(interaction_list, axis=1)
        X_test_df = X_test_df.drop(interaction_list, axis=1)
    print(f"Number of Parameters: {len(X_train_df.columns)}")
    
    model = linear_model.LinearRegression(fit_intercept=False) 
    cross_validation(model, X_train_df, y_train)
    if test:
        model.fit(X_train_df, y_train)
        y_train_pred = model.predict(X_train_df)
        print(X_test_df.shape)
        y_test_pred = model.predict(X_test_df)
        parity_plot(y_train, y_train_pred.flatten(), r_squared =model.score(X_train_df, y_train), title="Training Data", color="grey", alpha=0.5)
        parity_plot(y_test, y_test_pred.flatten(), r_squared =model.score(X_test_df, y_test), title="Test Data", color="blue", alpha=1)
        
    return X_train, X_test, y_train, y_test
    
    

@widgets.interact(degrees=(1,4))
def run(degrees=1, 
        interactions=False,
        Start = True,
        End = True,
        G = True,
        U = True,
        bi = True,
        uni = True,
        duplex = True,
        Pos1 = True,
        Pos2 = True,
        Pos6 = True,
        Pos13 = True,
        Pos14 = True,
        Pos18 = True,
        Dif_5_3 = True,
        Content_plus = True,
        Content_minus = True,
        Cons_plus = True,
        Cons_minus = True,
        Cons_Sum = True,
        Hyb19 = True,
        target = True,
        test=False):
    
    response="Activity"
    features = []
    if Start: features.append('Start')
    if End: features.append('End')
    if G: features.append('G')
    if U: features.append('U')
    if bi: features.append('bi')
    if uni: features.append('uni')
    if duplex: features.append('duplex')
    if Pos1: features.append('Pos1')
    if Pos2: features.append('Pos2')
    if Pos6: features.append('Pos6')
    if Pos13: features.append('Pos13')
    if Pos14: features.append('Pos14')
    if Pos18: features.append('Pos18')
    if Dif_5_3: features.append('Dif_5-3')
    if Content_plus: features.append('Content+')
    if Content_minus: features.append('Content-')
    if Cons_plus: features.append('Cons+')
    if Cons_minus: features.append('Cons-')
    if Cons_Sum: features.append('Cons_Sum')
    if Hyb19: features.append('Hyb19')
    if target: features.append('target')  
    X=num_dataset[features]
    y=num_dataset[response]
    

    do_the_thing(X,y,degrees,interactions, test)


# Question 11-12: Model selection and regularization
Great, you've built a model and selected which features to include! Next we will look at regularizing the model. The first plot shows you the magnitude of each of your parameters with different values of lambda using ridge regression. Try out a few different values.

The second plot gives you the train and validation results from running the model with three different values of lambda. When deciding which model to select, do we care about training or validation performance more?



In [ ]:
n_samples, n_features = 50, 10
X = np.random.randn(n_samples, n_features)
coef = 10 * np.random.randn(n_features)
y = np.dot(X, coef) + np.random.randn(n_samples)

# Define the regularization parameter values
lambda_val = [0, 1, 10, 100, 1000]

# Define the Ridge regression model
ridge = linear_model.Ridge()

@widgets.interact_manual(lambda_val=lambda_val)
def plot_ridge(lambda_val):
    ridge.alpha = lambda_val
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.set_ylim(-25, 25)
    ax.set_xlabel('Coefficient Index (Betas)')
    ax.set_ylabel('Coefficient Magnitude')
    ax.set_title('Effect of Regularization on Coefficient Magnitude')

    # Fit the Ridge model to the data and plot the coefficients
    ridge.fit(X, y)
    ax.plot(ridge.coef_)

    coef_sum = np.sum(np.abs(ridge.coef_))
    ax.text(0.5, 0.95, f'Sum of Absolute Parameter Values: {coef_sum:.3f}', transform=ax.transAxes, ha='center')

    plt.show()





In [ ]:
n_samples = 50
n_features = 40

# Define a function to generate sample data
def generate_data(n_samples=n_samples, n_features=n_features, noise_std=13):
    np.random.seed(0)
    X = np.random.randn(n_samples, n_features)
    coef = 10 * np.random.randn(n_features)
    y = np.dot(X, coef) + noise_std * np.random.randn(n_samples)
    return X, y

# Generate some sample data
X, y = generate_data()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Define the regularization parameter values
model = {"model 1 (low lambda)": 0, "model 2 (mid lambda)": 1, "model 3 (high lambda)": 10}

# Define the Ridge regression model
ridge = linear_model.Ridge()

@widgets.interact_manual(model=model)
def plot_ridge(model):
    ridge.alpha = model

    # Fit the Ridge model to the training data
    ridge.fit(X_train, y_train)

    # Evaluate the model on the training and test data
    y_train_pred = ridge.predict(X_train)
    y_test_pred = ridge.predict(X_test)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    # Plot the train and test R^2 values
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.bar(['Train', 'Validation'], [train_r2, test_r2])
    ax.set_ylim(0, 1.05)
    ax.set_title(f'$R^2$ with {n_features} features and {n_samples} samples')
    ax.set_xlabel('Data Split')
    ax.set_ylabel('$R^2$')
    ax.text(0, train_r2+0.01, f'{train_r2:.2f}')
    ax.text(1, test_r2+0.01, f'{test_r2:.2f}')
    plt.show()

# Optional: Other supervised approaches

Congratulations, you are now equipped with enough knowledge to evaluate other types of models rather than solely linear regression (Be careful though, the assumptions and hyperparameters from these other models are __much__ more complicated than MLR, but understanding MLR does equip you to understand these other models). I have proposed three different models, with some minor optimization. (You don't need to know the details, but I am happy to explain if you want to know more)
- Random Forest: an ensemble of 100 boosted decision trees, averaged to get a final result.
- Support Vector Machines: Similarly to polynomial features, we put the features into a different parameter space, and then perform a regression on those new features. In this case we are using a Radial Basis Function. 
- Neural Network: Using a bunch of manipulations, neural networks learn features from the original features, and fit a new model. It is based on using Rectified Linear Units.

It's okay if that sounds intimidating. Anytime I learn a new ML algorithm it is still intimidating. That being said, we can still use these models and evaluate their performance in the same way as we've been doing. 
__Note: Even if we can evaluate these models, without understanding the underlying math we cannot understand the ramification of these assumptions on interpreting parameters__. 


In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
from sklearn import ensemble, neural_network, svm

def rf(X,y):
    print('-'*30)
    print("Random Forest (boosted trees)")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = ensemble.GradientBoostingRegressor(learning_rate=.05, alpha=0.2)
    cross_validation(model, X_train, y_train)
    model.fit(X_train,y_train)
    r2_test = model.score(X_test,y_test)
    print(f"\nTest R2: {r2_test:.2f}")
    model.fit(X,y)
    r2 = model.score(X,y)
    y_pred = model.predict(X )
    parity_plot(y, y_pred, r_squared=r2,   title="RF, all data")
    
def svr(X,y):
    print('-'*30)
    print("Support Vector Regression")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = svm.SVR(kernel="rbf",gamma="scale", C=1000)
    cross_validation(model, X_train, y_train)
    model.fit(X_train,y_train)
    r2_test = model.score(X_test,y_test)
    print(f"\nTest R2: {r2_test:.2f}")
    model.fit(X,y)
    r2 = model.score(X,y)
    y_pred = model.predict(X )
    parity_plot(y, y_pred, r_squared=r2,  title="SVR, all data")
    
def nn(X,y):
    print('-'*30)
    print("Neural Network Regression (Multi-layer perceptron)")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = neural_network.MLPRegressor(hidden_layer_sizes=(30,30),
                                        learning_rate='invscaling',
                                        activation="relu", 
                                        max_iter=1000)
    cross_validation(model, X_train, y_train)
    model.fit(X_train,y_train)
    r2_test = model.score(X_test,y_test)
    print(f"\nTest R2: {r2_test:.2f}")
    model.fit(X,y)
    r2 = model.score(X,y)
    y_pred = model.predict(X )
    parity_plot(y, y_pred, r_squared=r2, title="NN, all data")
    
features = ['bi', 'uni', 'duplex', 
            'Pos1', 'Pos2', 'Pos6', 'Pos13', 'Pos14', 'Pos18', 
            'Dif_5-3', 'Content+', 
            'Cons+', 'Cons-', 'Cons_Sum', 'Hyb19', 'target']

X= num_dataset[features]
y= num_dataset["Activity"]

rf(X,y)
svr(X,y)
nn(X,y)